<a href="https://colab.research.google.com/github/Shreyanshd23/-Stock-Prediction-System-/blob/main/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from agent import Agent


In [2]:
!ls


 agent.py   functions.py  '^GSPC.csv'   __pycache__   sample_data


Agent


Import and Install


In [10]:
# 1) IMPORTS
from agent import Agent
import numpy as np
import math
import os

# 2) FUNCTIONS
def formatPrice(n):
    return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def getStockDataVec(filename):
    vec = []
    with open(filename + ".csv", "r") as f:
        lines = f.read().splitlines()
        for line in lines[1:]:
            vec.append(float(line.split(",")[4]))
    return vec

def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + list(data[0:t + 1])
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])

# 3) LOAD DATA
stock_name = "^GSPC"   # <-- your CSV must be named ^GSPC.csv in /content
data = getStockDataVec(stock_name)

# 4) TRAINING
window_size = 10
episode_count = 10  # Keep small for testing
batch_size = 32

agent = Agent(window_size)
l = len(data) - 1
profits_per_episode = []

os.makedirs("models", exist_ok=True)

for e in range(episode_count + 1):
    print(f"Episode {e}/{episode_count}")
    state = getState(data, 0, window_size + 1)
    total_profit = 0
    agent.inventory = []

    for t in range(l):
        action = agent.act(state)
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0

        if action == 1:  # Buy
            agent.inventory.append(data[t])
            print(f"Buy: {formatPrice(data[t])}")

        elif action == 2 and len(agent.inventory) > 0:  # Sell
            bought_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print(f"Sell: {formatPrice(data[t])} | Profit: {formatPrice(data[t] - bought_price)}")

        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            print("-" * 30)
            print(f"Total Profit: {formatPrice(total_profit)}")
            print("-" * 30)

    profits_per_episode.append(total_profit)

    if len(agent.memory) > batch_size:
        agent.expReplay(batch_size)

    if e % 10 == 0:
        model_filename = f"models/model_ep{e}.keras"
        agent.model.save(model_filename)
        print(f"Model saved to {model_filename}")

print("Training complete!")


Streaming output truncated to the last 5000 lines.
Buy: $805.22
Sell: $840.24 | Profit: -$78.97
Sell: $827.50 | Profit: -$71.45
Buy: $831.95
Buy: $874.09
Sell: $825.88 | Profit: -$26.42
Buy: $825.44
Sell: $838.51 | Profit: -$12.24
Buy: $845.85
Sell: $868.60 | Profit: $9.48
Buy: $827.16
Buy: $833.74
Buy: $835.19
Buy: $778.94
Buy: $773.14
Buy: $764.90
Sell: $752.83 | Profit: -$47.20
Buy: $735.09
Buy: $700.82
Sell: $696.33 | Profit: -$155.48
Buy: $712.87
Buy: $682.55
Buy: $676.53
Buy: $719.60
Buy: $756.55
Sell: $784.04 | Profit: -$73.35
Buy: $822.92
Buy: $806.12
Buy: $832.86
Buy: $815.94
Sell: $787.53 | Profit: -$108.71
Buy: $811.08
Sell: $842.50 | Profit: $26.29
Sell: $835.48 | Profit: -$74.22
Sell: $856.56 | Profit: -$17.03
Buy: $858.73
Sell: $852.06 | Profit: -$35.82
Sell: $865.30 | Profit: -$7.50
Buy: $869.60
Sell: $832.39 | Profit: -$37.03
Buy: $850.08
Sell: $843.55 | Profit: -$47.09
Buy: $857.51
Buy: $855.16
Buy: $873.64
Sell: $877.52 | Profit: -$25.73
Buy: $907.24
Sell: $919.53 | P